In [1]:
import os

from ladybug.sql import SQLiteResult
from ladybug import analysisperiod as ap

from eppy import *

import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

import pandas as pd

import numpy as np

from collections import OrderedDict

import datetime as dt

from IPython.display import display



In [2]:
root = os.getcwd()
root

'/Users/julietnwagwuume-ezeoke/Documents/cee256_local/_final_project_local'

In [3]:
def calc_metrics(simulated, measured):
    ts = simulated
    tm = measured
    tm_avg = np.mean(measured)
    n = len(tm)
    rmse = (100/tm_avg) * np.sqrt( (1/n) * np.sum((ts - tm)**2) )
    mbe = (100/tm_avg) * (np.sum(ts - tm)/n)
    return np.round(rmse,3), np.round(mbe,3)

In [4]:
def monthly_by_hour_from_sql(sql_dir_name):
    sqlFile = os.path.join(sql_dir_name, "eplusout.sql")
    sqld = SQLiteResult(sqlFile)

    elect = sqld.data_collections_by_output_name("Electricity:Facility")[-1]
    hot = sqld.data_collections_by_output_name("DistrictHeating:Facility")[-1]
    chill = sqld.data_collections_by_output_name("DistrictCooling:Facility")[-1]

    sim_month_by_hr = OrderedDict({
    "elect": elect.average_monthly_per_hour(),
    "hot": hot.average_monthly_per_hour(),
    "chill": chill.average_monthly_per_hour()
    })

    sim_monthly= OrderedDict({
    "elect": elect.total_monthly(),
    "hot": hot.total_monthly(),
    "chill": chill.total_monthly(),
    })

    return sim_month_by_hr, sim_monthly

In [5]:
# new_dir_name = os.path.join(adjusted_model_dir, "03-02_02-Hourly-Meter")
sim_names = {
    3: "03-06_00-eh-6",
}

adjusted_model_dir = os.path.join(root, "eppy_adjusted_models")
sim_name_paths = {k: os.path.join(adjusted_model_dir, v) for k,v in sim_names.items()}

In [6]:
sim_data_hr = OrderedDict()
sim_data_month = OrderedDict()

for k, v in zip(sim_names.values(), sim_name_paths.values()):
    sim_data_hr[k] = monthly_by_hour_from_sql(v)[0]
    sim_data_month[k] = monthly_by_hour_from_sql(v)[1]

# Load and Convert Historical Data

## Temperature

In [7]:
temp_data_path = "/Users/julietnwagwuume-ezeoke/My Drive/CEE256_BuildSys/final_256/Data/2019 Palo Alto weather_calc.xlsx"
temp_data_hourly = pd.read_excel(temp_data_path, sheet_name="weather", usecols="A:L")
temp_data_hourly["Time"] = temp_data_hourly["Date and time"].str[-8:]
temp_data_hourly

,month,date,Date and time,Dry Bulb Temperature (F),Wet Bulb Temperature (F),Atmospheric Pressure (atm),Relative Humidity (%),Dew Point Temperature (F),Global Solar (Btu/ft2),Normal Solar (Btu/ft2),Diffuse Solar (Btu/ft2),Wind Speed (mph),Time
0,1,2019/01/01,2019/01/01 @ 00:00:00,48.74,39.24,1.00,40,26.27,0.0,0.0,0.0,15.21,00:00:00
1,1,2019/01/01,2019/01/01 @ 01:00:00,48.02,38.84,1.01,41,26.22,0.0,0.0,0.0,13.20,01:00:00
2,1,2019/01/01,2019/01/01 @ 02:00:00,47.30,38.28,1.01,41,25.65,0.0,0.0,0.0,11.18,02:00:00
3,1,2019/01/01,2019/01/01 @ 03:00:00,46.58,37.87,1.01,42,25.58,0.0,0.0,0.0,9.17,03:00:00
4,1,2019/01/01,2019/01/01 @ 04:00:00,45.86,37.45,1.01,43,25.50,0.0,0.0,0.0,6.93,04:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,12,2019/12/31,2019/12/31 @ 19:00:00,47.84,44.59,1.00,78,41.36,0.0,0.0,0.0,1.12,19:00:00
8756,12,2019/12/31,2019/12/31 @ 20:00:00,48.02,45.06,1.00,80,42.19,0.0,0.0,0.0,6.93,20:00:00
8757,12,2019/12/31,2019/12/31 @ 21:00:00,47.30,44.68,1.00,82,42.13,0.0,0.0,0.0,5.82,21:00:00
8758,12,2019/12/31,2019/12/31 @ 22:00:00,46.58,44.15,1.00,83,41.74,0.0,0.0,0.0,5.82,22:00:00


In [8]:
# temp_data_hourly[]

In [9]:
temp_month_by_hour = temp_data_hourly.groupby(["month", "Time"])["Dry Bulb Temperature (F)"].mean()
temp_month_by_hour_c = (temp_month_by_hour -32) * (5/9)
temp_month_by_hour_c

month  Time    
1      00:00:00    10.103226
       01:00:00     9.764516
       02:00:00     9.464516
       03:00:00     9.148387
       04:00:00     8.841935
                     ...    
12     19:00:00    10.793548
       20:00:00    10.990323
       21:00:00    10.667742
       22:00:00    10.345161
       23:00:00     9.987097
Name: Dry Bulb Temperature (F), Length: 288, dtype: float64

## Historical

In [10]:
# conversion factors
tonHr_kwHr = 3.5169 # 1 ton-hr = 3.517 kWh
kbtu_kwHr = 1/3.412 # 1 kbtu = 0.29 kwHr

# load data 
historical_data_path = os.path.join(root, "data/Thornton Utilities 2019.xlsx")

# electricity # kwh 
hist_elect_hourly = pd.read_excel(historical_data_path, sheet_name="Electricity hourly", usecols="A:E")

# chilled water  -> convert ton hours to kWh
hist_chw_hourly = pd.read_excel(historical_data_path, sheet_name="CHW hourly", usecols="A:E")
hist_chw_hourly["kWh"] = hist_chw_hourly["ton-hours"]*tonHr_kwHr

# hot water -> convert kbtu to kWh
hist_hw_hourly = pd.read_excel(historical_data_path, sheet_name="HW hourly", usecols="A:E")
hist_hw_hourly["kWh"] = hist_hw_hourly["kBtu"]*kbtu_kwHr 

In [11]:
# month by hour
hist_month_by_hr = OrderedDict({
    "elect" : hist_elect_hourly.groupby(["Month", "Hour"])["kWh"].mean(),
    "hot": hist_hw_hourly.groupby(["Month", "Hour"])["kWh"].mean(),
    "chill": hist_chw_hourly.groupby(["Month", "Hour"])["kWh"].mean(),
})

# monthly 
hist_monthly = OrderedDict({
   "elect": hist_elect_hourly.groupby("Month")["kWh"].sum(),
   "hot":  hist_hw_hourly.groupby("Month")["kWh"].sum(),
   "chill":  hist_chw_hourly.groupby("Month")["kWh"].sum()
})

# Comparison of Hourly Trends

In [12]:
hours = [dt.datetime(2019, 1, 1, i ).strftime("%H:%M") for i in range(24)]
aperiods = {
    1:  ap.AnalysisPeriod(st_month=1, end_month=1),
    6 : ap.AnalysisPeriod(st_month=6, end_month=6),
    8 : ap.AnalysisPeriod(st_month=8, end_month=8)
}

Updated end_day from 31 to 30


In [13]:
def create_analysis_period(num):
    return ap.AnalysisPeriod(st_month=num, end_month=num)

In [24]:
def temp_vs_energy_use_by_month(month_num, month_name):

    month = create_analysis_period(month_num)
    blueShades = px.colors.qualitative.Vivid[:len(sim_data_hr.keys())]

    fig = make_subplots(
        rows=1, cols=3,
        subplot_titles=("Electricity", "Hot Water", "Chilled Water"))

    # historical/ measured data 
    for (ix, value) in  enumerate(hist_month_by_hr.values()):
        showLegend = True
        if ix > 0:
            showLegend = False
        fig.add_trace(go.Scatter(  
            x=temp_month_by_hour_c[month.st_month],
            y=value[month.st_month] , 
            mode='markers',
            name="Historical",
            legendgroup="Months",
            line=dict(color='red'),
            showlegend=showLegend),
            row=1, col=ix+1)

    # simulated datasets 
    for (name, dataset), blueShade in zip(sim_data_hr.items(), blueShades):
        for (ix, sim), hist in zip(enumerate(dataset.values()),hist_month_by_hr.values()) :
            showLegend = True
            if ix > 0:
                showLegend = False
            fig.add_trace(go.Scatter( 
                x=temp_month_by_hour_c[month.st_month],
                y=sim.filter_by_analysis_period(month).values , 
                mode='markers',
                name=name,
                legendgroup="Months",
                line=dict(color=blueShade),
                showlegend=showLegend),
                row=1, col=ix+1)

    fig.update_layout(title=f"{month_name} Average Hourly Energy Usage",
        yaxis_title="Energy (kWh)",)
   
   # calculate metrics 
    # metrics = {}
    # for name, dataset in sim_data_hr.items():
    #     type_dict = {}
    #     for (k, sim), hist in zip(dataset.items(), hist_month_by_hr.values()):
    #         type_dict[k] = calc_metrics(sim.filter_by_analysis_period(month).values, hist[month.st_month])
    #     metrics[name] = type_dict
    # metrics = pd.DataFrame(metrics)
    # display(metrics)

    fig.show()

In [25]:
temp_vs_energy_use_by_month(1, "Jan")

In [16]:
temp_vs_energy_use_by_month(6, "June")

Updated end_day from 31 to 30


In [17]:
temp_vs_energy_use_by_month(8, "July")

## Per Day...

In [18]:
def temp_vs_energy_use_by_day(month_num, month_name):

    month = create_analysis_period(month_num)
    blueShades = px.colors.qualitative.Vivid[:len(sim_data_hr.keys())]

    fig = make_subplots(
        rows=1, cols=3,
        subplot_titles=("Electricity", "Hot Water", "Chilled Water"))

    # historical/ measured data 
    for (ix, value) in  enumerate(hist_month_by_hr.values()):
        showLegend = True
        if ix > 0:
            showLegend = False
        fig.add_trace(go.Scatter(  
            x=temp_month_by_hour_c[month.st_month],
            y=value[month.st_month] , 
            mode='markers',
            name="Historical",
            legendgroup="Months",
            line=dict(color='red'),
            showlegend=showLegend),
            row=1, col=ix+1)

    fig.update_layout(title=f"{month_name} Average Hourly Energy Usage",
        yaxis_title="Energy (kWh)",)

    fig.show()

# Comparison of Annual Trends 

In [19]:
months = [dt.datetime(2019, i+1, 1).strftime("%b") for i in range(12)]

In [20]:
def plot_datasets_monthly():
    blueShades = px.colors.qualitative.Vivid[:len(sim_data_hr.keys())]

    fig = make_subplots(
        rows=1, cols=3,
        subplot_titles=("Electricity", "Hot Water", "Chilled Water"))

    # historical/ measured data 
    for (ix, value) in  enumerate(hist_monthly.values()):
        showLegend = True
        if ix > 0:
            showLegend = False
        fig.add_trace(go.Scatter(  
            x=months,
            y=value, 
            mode='lines',
            name="Historical",
            legendgroup="Months",
            line=dict(color='red'),
            showlegend=showLegend),
            row=1, col=ix+1)

    # simulated datasets 
    for (name, dataset), blueShade in zip(sim_data_month.items(), blueShades):
        for (ix, sim), hist in zip(enumerate(dataset.values()),hist_monthly.values()) :
            showLegend = True
            if ix > 0:
                showLegend = False
            fig.add_trace(go.Scatter( 
                x=months,
                y=sim.values , 
                mode='lines+markers',
                name=name,
                legendgroup="Months",
                line=dict(color=blueShade),
                showlegend=showLegend),
                row=1, col=ix+1)

    fig.update_layout(title=f"Monthly Total Energy Usage",
        yaxis_title="Energy (kWh)",)
   
   # calculate metrics 
    metrics = {}
    for name, dataset in sim_data_month.items():
        type_dict = {}
        for (k, sim), hist in zip(dataset.items(), hist_monthly.values()):
            type_dict[k] = calc_metrics(sim.values, hist)
        metrics[name] = type_dict
    metrics = pd.DataFrame(metrics)
    display(metrics)

    fig.show()

In [21]:
plot_datasets_monthly()

,03-06_00-eh-6
chill,"(68.056, 51.115)"
elect,"(34.158, -32.478)"
hot,"(84.888, -74.878)"


In [22]:
hist_monthly["hot"]

Month
1      9563.779652
2     11205.168864
3     10021.888756
4      5739.921388
5      8015.771756
6      2812.879287
7      1998.993427
8      1290.168084
9      1966.378451
10     4721.300882
11     7453.441033
12    10536.379511
Name: kWh, dtype: float64

In [23]:
# {'elect': 3.100677715042496, 'hot': 1.8389211816597943, 'chill': 9.354337655572776}
    